In [2]:
import math

In [3]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv
    
    if H_total < P /T:
        print("PASS")
    else:
        print("FAIL")
    print("Total dose: ", H_total, "mSv/wk")
    print("Dose limit: ", P / T, "mSv/wk")



In [4]:
# Vault #1 Wall A Secondary Barrier
# Worst case scenario of 33.37 deg barely grazing the edge of primary barrier
# 8.57258 m distance
verify_2barrier(0.02,(1/40),68.58,8.57258702,26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.6394931153452919 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk


In [5]:
# Vault #1 Wall C Secondary Barrier
# Worst case scenario of 33.92 deg barely grazing the edge of primary barrier
# 8.627792 m distance
verify_2barrier(0.02,(1/40),68.58,8.627792498,26,32,2.77e-3,2.53e-3)

PASS
Total dose:  0.6313356226374781 mSv/wk
Dose limit:  0.7999999999999999 mSv/wk


In [6]:
# Vault #1 Wall B Secondary Barrier
# Worst case scenario of 90 deg incidence
verify_2barrier(0.02,1,106.9,convert_to_meters(20,11.81056),17,19,4.26e-4,1.89e-4)

PASS
Total dose:  0.01994554665683441 mSv/wk
Dose limit:  0.02 mSv/wk


In [51]:
# maze width
maze_width = convert_to_meters(8,5.00242)
# maze height
maze_height = convert_to_meters(10,0)
# dz
dz = convert_to_meters(24,11.10850)
# dzz
dzz = convert_to_meters(33,11.99989)

test1 = dz / (maze_width * maze_height)**0.5

test2 = dzz / (maze_width * maze_height)**0.5

if test1 < 6 and test1 > 2:
    print('PASS')

if test2 < 6 and test2 > 2:
    print('PASS')

if 1 < maze_height / maze_width and maze_height / maze_width < 2:
    print('PASS')
    print(maze_height/maze_width)

print(f"maze height = {maze_height} m")
print(f"maze width = {maze_width} m")


PASS
PASS
PASS
1.188090344765997
maze height = 3.048 m
maze width = 2.5654614679999996 m


In [49]:
# H_s
# Variables for Vault #1
d_r = convert_to_meters(25,1.28538)
d_h = convert_to_meters(17,9.04728) + 1
area_0 = (0.40 * d_h)**2 # m

# Find A_z
width_Az = convert_to_meters(10,3.78273)
area_z = width_Az * maze_height


In [53]:
print(f"dz = {dz} m")
print(f"dzz = {dzz} m")
print(f"d_r = {d_r} m")
print(f"d_h = {d_h} m")
print(f"width_Az = {width_Az} m")
print("--------------------------")
print(f"area_0 = {area_0} m^2")
print(f"area_z = {area_z} m^2")

dz = 7.5973559 m
dzz = 10.363197206 m
d_r = 7.652648652000001 m
d_h = 6.4114009119999995 m
width_Az = 3.1440813420000002 m
--------------------------
area_0 = 6.57696986470311 m^2
area_z = 9.583159930416 m^2
